In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Getting top 100 songs from 'popvortex'

In [2]:
url="https://www.popvortex.com/music/charts/top-100-songs.php"
response = requests.get(url)
response.status_code

200

In [3]:
soup = BeautifulSoup(response.content, "html.parser")

In [4]:
#soup.select('body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content > p > cite')

In [5]:
name=[]
for i in soup.select('body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content > p > cite'):
    name.append(i.get_text().strip())


In [6]:
#soup.select('body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content > p > em')

In [7]:
artist=[]

for a in soup.select('body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content > p > em'):
    artist.append(a.get_text().strip())

In [8]:
top100=pd.DataFrame({'song':name,
                    'artist':artist})
top100

,song,artist
0,Margaritaville,Jimmy Buffett
1,Come Monday,Jimmy Buffett
2,Rich Men North of Richmond,Oliver Anthony Music
3,Cheeseburger In Paradise,Jimmy Buffett
4,All Star,Smash Mouth
...,...,...
95,"Don't Keep It In Your Heart, There's Nothing H...",Vy Napi
96,Spirit In the Sky,Norman Greenbaum
97,Snooze,SZA
98,Dial Drunk,Noah Kahan & Post Malone


In [9]:
top100['song'].str.startswith(' ').value_counts()

False    100
Name: song, dtype: int64

In [10]:
top100['song'].str.endswith(' ').value_counts()

False    100
Name: song, dtype: int64

In [11]:
top100['artist'].str.startswith(' ').value_counts()

False    100
Name: artist, dtype: int64

In [12]:
top100['artist'].str.endswith(' ').value_counts()

False    100
Name: artist, dtype: int64

In [32]:
#top100.to_csv('top100.csv',index=False)

### Eurovision winner songs and artist (1956-2023)

In [13]:
url="https://en.wikipedia.org/wiki/List_of_Eurovision_Song_Contest_winners"
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.content, "html.parser")

In [14]:
#mw-content-text > div.mw-parser-output > table:nth-child(13) > tbody

In [15]:
soup.select('#mw-content-text > div.mw-parser-output > table:nth-child(41) > tbody')

[]

In [16]:
songs_tab=soup.select('table')[0]

In [17]:
#songs_tab.select("td:nth-of-type(2) a")

In [18]:
songs=[]
for s in songs_tab.select("td:nth-of-type(2) a"):
    songs.append(s.get('href'))

In [19]:
len(songs)

71

In [20]:
import random
from time import sleep
songs_soup=[]

for song in songs:
    # send request
    url = "https://en.wikipedia.org" + song
    response = requests.get(url)
    #print(song, response.status_code)

    # parse & store html
    soup = BeautifulSoup(response.content, "html.parser")
    songs_soup.append(soup.select("table.infobox"))

    # respectful nap:
    wait_time = random.randint(1,4000)
    sleep(wait_time/1000)

In [21]:
songs = []
artists = []

for song in songs_soup:
    try:
        songs.append(song[0].select("th.infobox-above")[0].get_text())
    except:
        songs.append("NA")
    try:
        artists.append(song[0].find("th", string = "Artist(s)").parent.select("a")[0].get_text())
    except:
        try:
            artists.append(song[0].select("th.infobox-header")[0].select("a")[1].get_text())
        except:
            try:
                artists.append(song[0].find("th", string = "As").parent.select("a")[0].get_text())
            except:
                try:
                    artists.append(song[0].find("th", string = "Artist(s)").parent.select("div")[0].get_text())
                except:
                    artists.append("NA")

In [22]:
euro_winners = pd.DataFrame({"song":songs,
                             "artist":artists})

In [23]:
euro_winners

,song,artist
0,"""Refrain""",Lys Assia
1,"""Net als toen""",Corry Brokken
2,"""Dors, mon amour""",André Claveau
3,"""'n Beetje""",Teddy Scholten
4,"""Tom Pillibi""",Jacqueline Boyer
...,...,...
66,"""Arcade""",Duncan Laurence
67,NA,NA
68,"""Zitti e buoni""",Måneskin
69,"""Stefania""",Kalush Orchestra


In [24]:
euro_winners= euro_winners.drop([euro_winners.index[67]]).reset_index(drop=True)

In [25]:
euro_winners['song']=euro_winners['song'].replace('"','',regex=True)
euro_winners

,song,artist
0,Refrain,Lys Assia
1,Net als toen,Corry Brokken
2,"Dors, mon amour",André Claveau
3,'n Beetje,Teddy Scholten
4,Tom Pillibi,Jacqueline Boyer
...,...,...
65,Toy,Netta
66,Arcade,Duncan Laurence
67,Zitti e buoni,Måneskin
68,Stefania,Kalush Orchestra


In [26]:
euro_winners.song=euro_winners.song.replace(r'^ | $','',regex=True)

In [27]:
euro_winners['artist'].str.startswith(' ').value_counts()

False    70
Name: artist, dtype: int64

In [28]:
euro_winners['artist'].str.endswith(' ').value_counts()

False    70
Name: artist, dtype: int64

#### Merging the top 100 and eurovision winners into one dataframe

In [33]:
song_recommend=pd.concat([top100,euro_winners],axis=0)

In [34]:
print(song_recommend.shape)
song_recommend.isna().sum() #to check if concat worked correctly

(170, 2)


song      0
artist    0
dtype: int64

#### Saving the dataframe to a csv for building recommendation model

In [ ]:
song_recommend.to_csv(r'.\datasets\top_songs.csv',index=False)